# En este notebook vamos a ilustrar el proceso de creación de un RAG multimodal, para la mejora de precisión en el proyecto de Wivboost de embeddings de los videos.

# **Importamos las dependencias**

In [ ]:
# %pip install rich -q

# Fijate en las versiones de las librerías que se instalan, son para que no haya problemas de compatibilidad
# %pip install --upgrade google-cloud-aiplatform==1.71.1 vertexai==1.71.1 pymupdf rich colorama -q

In [3]:
# --- Importaciones para Manejo de Archivos y Directorios ---
import os
import sys
import subprocess
from dotenv import load_dotenv


# --- Importamos Vertex AI ---
import vertexai


# --- Importaciones para Visualización y Formato de Texto ---

# Se utiliza para mostrar contenido enriquecido, como texto con formato Markdown,
# directamente en entornos como Jupyter Notebooks o IPython.
from IPython.display import Markdown, display

# Importa la clase `Markdown` de la biblioteca `rich`, que sirve para renderizar
# Markdown con formato avanzado en la terminal. Se le da un alias `RichMarkdown`
# para evitar conflictos de nombre con la importación anterior.
from rich.markdown import Markdown as RichMarkdown


# --- Importaciones para el Modelo Generativo de Vertex AI ---

# Importa las clases necesarias del SDK de Vertex AI para interactuar con los modelos generativos.
# - GenerationConfig: Para configurar los parámetros de la respuesta (ej. temperatura, top_p).
# - GenerativeModel: La clase principal para cargar y usar un modelo generativo como Gemini.
# - Image: Para manejar y enviar imágenes como parte de la entrada al modelo (enfoque multimodal).
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

# --- Inicialización del Modelo de Lenguaje (LLM) ---

# Crea una instancia de un modelo generativo, cargando específicamente el modelo 'gemini-2.0-flash'.
# Este objeto `modelo` se usará para enviar prompts (instrucciones) y recibir respuestas del LLM.
text_model = GenerativeModel('gemini-2.0-flash')
multimodal_model = text_model
multimodal_model_flash = text_model

In [11]:
# --- Descargamos otras deoendencias extra que nos marca el repo, además de el archivo ---
!gsutil -m rsync -r gs://github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_old_version .
print("Download completed")

Building synchronization state...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Starting synchronization...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_old_version/class_a_share.png...
Copying gs://github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_old_version/data/google-10k-sample-part2.pdf...
Copying gs://

Usan estos datos: [Google-10k](https://abc.xyz/assets/investor/static/pdf/20220202_alphabet_10K.pdf)

# **Configuración de las credenciales**

In [ ]:
# --- Carga las Variables de Entorno ---
load_dotenv()

# Ejecutamos los  comandos de terminal para obtener el ID del proyecto de Google Cloud
PROJECT_ID = subprocess.check_output(['gcloud', 'config', 'get-value', 'project'], text = True).strip()

# Verificamos
print(f'Project ID: {PROJECT_ID}')

Project ID: spry-byway-462510-i4


In [10]:
# --- Iniciamos vertex AI ---
vertexai.init(project = PROJECT_ID, location = 'us-central1')

# **Extraemos la metadata del documento**

In [1]:
# Importamos una dependencia adicional para obtener la metadata
from intro_multimodal_rag_utils import get_document_metadata

In [13]:
# Aquí están los pdf que vamos a utilizar para el ejemplo
pdf_folder_path = 'Jupyter/Ejemplo_RAG/data'


# Hacemos el prompt para las imagenes (originalmente estaba en inglés, lo traduje al español)
image_description_prompt ="""
Explica lo que está pasando en la imagen.

Si es una tabla, extraiga todos los elementos de la tabla.

Si es un gráfico, explique los hallazgos en el gráfico.

No incluya ningún número que no se mencione en la imagen.
"""

# Extraemos el texto y las imágenes de los PDFs
text_metadata_df, image_metadata_df = get_document_metadata(

    multimodal_model,
    pdf_folder_path,
    image_save_dir = 'Jupyter/Ejemplo_RAG/images',
    image_description_prompt = image_description_prompt,
    embedding_size = 1408

)

print("---- Proceso completado ----")

---- Proceso completado ----


In [14]:
text_metadata_df

""


In [18]:
print(os.listdir('/Users/wivboost/Desktop/Embeddings_OFF/Jupyter/Ejemplo_RAG/images'))

['class_a_share.png', 'tac_table_revenue.png']
